In [18]:
# Import dependencies
from citipy import citipy
import numpy as np
import requests
import pandas as pd
import gmaps

# Google developer API key
from config import g_key

# Access maps with unique API key
gmaps.configure(api_key=g_key)

In [19]:
# Import Weather.csv
df = pd.read_csv('Weather.csv')
df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Dikson,73.5069,80.5464,262.53,89,95,8.13,RU,1634968857
1,1,New Norfolk,-42.7826,147.0587,286.29,84,100,0.74,AU,1634969004
2,2,Slave Lake,55.2834,-114.7690,278.43,56,100,4.32,CA,1634969005
3,3,Hasaki,35.7333,140.8333,291.37,47,32,8.61,JP,1634968833
4,4,Aleksandrovsk-Sakhalinskiy,50.8975,142.1561,278.34,73,86,5.05,RU,1634969007


In [20]:
# Customize the size of the figure
figure_layout = {
    'width': '800px',
    'height': '600px'
}
fig = gmaps.figure(layout=figure_layout)

# Create heatmap
heatmap_layer = gmaps.heatmap_layer(locations = coordinates,weights = humidity)
fig.add_layer(heatmap_layer)
fig

NameError: name 'coordinates' is not defined